In [1]:
import time, datetime
from collections import defaultdict

from fast_trader.dtp import dtp_api_id
from fast_trader.dtp import type_pb2 as dtp_type
from fast_trader.dtp_quote import TradeFeed, OrderFeed, TickFeed
from fast_trader.strategy import Strategy, StrategyFactory, to_timeint
from fast_trader.utils import timeit, message2dict, int2datetime, attrdict

/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [2]:

class DemoStrategy(Strategy):
    """
    Demo
    """
    
    strategy_id = 8
    strategy_name = 'Demo strategy'

    @property
    def now(self):
        return datetime.datetime.now()

    @property
    def holdings(self):
        positions = self.get_positions()
        return [p['code'] for p in positions]

    def on_start(self):
        """
        执行策略启动前的一些初始化操作
        """
        pass

    def on_market_trade(self, data):
        """
        响应逐笔成交行情
        """

        # 过滤撤单记录
        if data.nTurnover == 0:
            return

        if data.nTime < 93000000:  # 不参与集合竞价
            return

    def on_market_snapshot(self, data):
        """
        响应快照行情
        """
        pass

    def on_order(self, order):
        """
        响应报单回报
        """
        print('\n-----报单回报-----')
        print(order)

    def on_trade(self, trade):
        """
        响应成交回报
        """
        print('\n-----成交回报-----')
        print(trade)

    def on_order_cancelation(self, data):
        """
        响应撤单回报
        """
        print('\n-----撤单回报-----')
        print(data)



if __name__ == '__main__':


    factory = StrategyFactory()
    strategy = factory.generate_strategy(
        DemoStrategy,
        trader_id=1,
        strategy_id=DemoStrategy.strategy_id
    )
    
    # 订阅行情
    tf = TradeFeed()
    tf.subscribe(['600056'])

    of = OrderFeed()
    of.subscribe(['600056'])

    tk = TickFeed()
    tk.subscribe(['600056', '601668', '601555'])

    strategy.add_datasource(tf)
    strategy.add_datasource(tk)

    strategy.start()

    ## 下单
    # strategy.buy('601555', '16', 500)
    ## 全撤
    # strategy.cancle_all()



2019-04-24 16:10:42,212 - MainThread - trader - WARNING - 初始请求编号 策略=8 1753936220
Login success


/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [3]:
strategy.get_capital()

/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'account_no': '011000106328',
 'available': '3999957953.170',
 'balance': '4000000000.000',
 'freeze': '42046.830',
 'securities': '139234831.000',
 'total': '4139192784.170'}

In [4]:
strategy.buy('601555', '16', 500)

16 <class 'str'>


/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'order_side': 1,
 'code': '601555',
 'price': '16',
 'quantity': 500,
 'exchange': 1,
 'order_original_id': '1753936220',
 'status': 0,
 'placed_localtime': 161042367}


-----报单回报-----
{'order_exchange_id': '0', 'placed_time': '', 'freeze_amount': 0.0, 'status': 9, 'order_original_id': '1753936220', 'account_no': '011000106328', 'exchange': 1, 'code': '601555', 'quantity': 500, 'order_side': 1, 'price': 16.0, 'message': '闭市期间,系统禁止委托'}
